In [36]:
from copy import deepcopy

from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
from openpyxl.cell import Cell

redFill = PatternFill(start_color='FFFF0000',
                   end_color='FFFF0000',
                   fill_type='solid')
blueFill = PatternFill(start_color='00FFFF',
                   end_color='00FFFF',
                   fill_type='solid')

## Low-level functions ##
"""
Returns string of hh-mm-ss
"""
from time import gmtime, strftime
def getTime():
    return strftime("%H-%M-%S", gmtime())

"""
Splits input string by delimiter
Returns list of these strings 
"""
import re
def split(string):
    return re.split(', |; |\. |  ', string)

"""
Add column with to sheet to max column of sheet
Labels first row with column_header
"""
def add_column(sheet, column_header):
    max_col = sheet.max_column
    sheet.cell(row=1, column=max_col+1).value = column_header

"""
Returns True if statement contains any of words in keywords
Else return False
"""
def contains(statement, keywords):
    for word in keywords:
        ## Edge cases

        if word == "id":
            pattern1 = re.compile(".*\w+id.*")
            pattern2 = re.compile(".*id\w+.*")
            if not (pattern1.match(statement) or pattern2.match(statement)):
                if word in statement:
                    return True

        elif word == "ied":
            pattern1 = re.compile(".*\w+ied.*")
            pattern2 = re.compile(".*ied\w+.*")
            if not (pattern1.match(statement) or pattern2.match(statement)):
                if word in statement:
                    return True
            
        elif word == "tic":
            pattern1 = re.compile(".*\w+tic.*")
            pattern2 = re.compile(".*tic\w+.*")
            if not (pattern1.match(statement) or pattern2.match(statement)):
                if word in statement:
                    return True

        elif word == "psychos":
            if word in statement:
                if not "psychosomatic" in statement: 
                    return True

        elif word == "psychosomatic":
            if word in statement:
                return True

        ## Regular word
        elif word in statement:
            return True

    return False

"""
Shortens diagnostic_type
admission => addx
discharge => dcdx
"""
def get_shortened_diagnostic_type(diagnostic_type):
    if diagnostic_type == "admission":
        return "addx"
    elif diagnostic_type == "discharge":
        return "dcdx"
    else:
        print("WARNING: BAD DIAGNOSTIC TYPE: {}".format(diagnostic_type))
        return ""

"""
Create a dictionary of column names
"""   
def get_column_names(sheet):
    column_names = {}
    i = 0
    for col in sheet.iter_cols(1, sheet.max_column):
        column_names[col[0].value] = i
        i += 1
    return column_names

"""
Create key list from dictionary
"""
def get_key_list(dictionary):
    key_list = []
    for i in dictionary.keys():
        key_list.append(i)
    return key_list

def manipulate_illness_list(illnesses, full_diagnosis):
    illnesses = deepcopy(illnesses)
    ## ASD/FASD 
    if "ASD" in illnesses and "FASD" in illnesses:
        illnesses.remove("ASD")

    ## EOS/Personality
    if "Schizophrenia Spectrum and Other Psychotic Disorders" in illnesses and "Personality" in illnesses:
        illnesses.remove("Schizophrenia Spectrum and Other Psychotic Disorders")      
    
    return illnesses

def get_difference_list(short_list, long_list):
    short_list = deepcopy(short_list)
    long_list = deepcopy(long_list)

    for x in short_list:
        long_list.remove(x)

    return long_list

## High-level Functions ##
"""
For each illness in diagnosis, label the corresponding column with a 0
"""
def one_hot_encode_diagnoses(sheet, diagnostic_type, ignore_keywords, illness_keywords):
    ## Setup diagnostic type
    diagnostic_type_shortened = get_shortened_diagnostic_type(diagnostic_type)
        
    for row_cells in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        
        ## Ensure valid cell data
        if type(row_cells[column_names['{}_diagnosis'.format(diagnostic_type)]].value) is str:
            
            ## Get admission diagnosis in lower-case
            full_diagnosis = row_cells[column_names['{}_diagnosis'.format(diagnostic_type)]].value.lower()
            diagnoses = split(full_diagnosis)
            illnesses = []
            
            ## Add illnesses for each statement in diagnosis
            for statement in diagnoses:
                
                    ## Add all illnesses associated with each statement
                    for illness in illness_keywords:
                        if not contains(statement, ignore_keywords) and contains(statement, illness_keywords[illness]):
                            illnesses.append(illness)
                            row_cells[column_names['{}_{}'.format(diagnostic_type_shortened, illness)]].value = 1

                    ## Exceptions (REMOVES SUICIDE IF ONLY 2, should it?)
                    manipulated_illnesses = manipulate_illness_list(illnesses, full_diagnosis)
                    difference_list = get_difference_list(manipulated_illnesses, illnesses)
                    for difference in difference_list:
                        row_cells[column_names['{}_{}'.format(diagnostic_type_shortened, difference)]].value = 0
                    if len(difference_list) > 0:
                        print("*****************")
                        print(illnesses)
                        print()
                        #row_cells[column_names['{}_{}'.format(diagnostic_type_shortened, difference)]].fill = blueFill
                                
"""
Find first illness in diagnosis, then label the main diagnosis
"""                           
def set_main_diagnosis(sheet, diagnostic_type, ignore_keywords, illness_keywords):
    ## Setup diagnostic type
    diagnostic_type_shortened = get_shortened_diagnostic_type(diagnostic_type)
        
    for row_cells in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        
        ## Ensure valid cell data
        if type(row_cells[column_names['{}_diagnosis'.format(diagnostic_type)]].value) is str:

            ## Get diagnosis in lower-case
            full_diagnosis = row_cells[column_names['{}_diagnosis'.format(diagnostic_type)]].value.lower()
            diagnoses = split(full_diagnosis)
            illnesses = []

            ## Loop through diagnoses until illness is found
            while len(diagnoses) > 0:
                main_diagnosis = diagnoses[0]

                ## Get main illness
                for illness in illness_keywords:
                    if not contains(main_diagnosis, ignore_keywords) and contains(main_diagnosis, illness_keywords[illness]):
                        illnesses.append(illness)
                    
                ## Update spreadsheet with illness
                if len(illnesses) == 1:
                    row_cells[column_names["main_{}".format(diagnostic_type_shortened)]].value = illnesses[0]
                    break
                    
                ## if more than 1, will show list in red
                elif len(illnesses) > 1:
                    
                    illnesses = manipulate_illness_list(illnesses, full_diagnosis)
                        
                    if len(illnesses) > 1:
                        row_cells[column_names["main_{}".format(diagnostic_type_shortened)]].value = str(illnesses)
                        row_cells[column_names["main_{}".format(diagnostic_type_shortened)]].fill = redFill
                        print("***************")
                        print(illnesses)
                        print(main_diagnosis)
                    elif len(illnesses) == 1:
                        row_cells[column_names["main_{}".format(diagnostic_type_shortened)]].value = illnesses[0]
                    else:
                        print("EDGE CASE")

                    break
                ## remove first element of diagnoses
                diagnoses = diagnoses[1:]
            
            ## If none found, fill with color
            if len(illnesses) == 0:
                row_cells[column_names["main_{}".format(diagnostic_type_shortened)]].fill = redFill
"""
Set all columns with binary headers to 0
"""
def set_binaries_to_zero(sheet, diagnostic_type, illnesses):
    ## Setup diagnostic type
    diagnostic_type_shortened = get_shortened_diagnostic_type(diagnostic_type)
    
    for row_cells in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        for illness in illnesses:
            row_cells[column_names["{}_{}".format(diagnostic_type_shortened, illness)]].value = 0
            
"""
Set column headers at end of columns
"""
def set_headers(sheet, diagnostic_type, illnesses):
    ## Setup diagnostic type
    diagnostic_type_shortened = get_shortened_diagnostic_type(diagnostic_type)
    
    add_column(sheet, "main_{}".format(diagnostic_type_shortened))
    for illness in illnesses:
        add_column(sheet, "{}_{}".format(diagnostic_type_shortened, illness))
        
"""
Read in sheet of illness keywords
Return dictionary mapping illnesses to list of keywords
"""
def read_in_illness_keywords(sheet, sheet_columns):
    illness_keywords = {}
    for row_cells in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        illness = row_cells[sheet_columns['Illness']].value
        illness_keywords[illness] = []
        for col in range(sheet_columns['Illness']+1, sheet.max_column):
            if type(row_cells[col].value) is str:
                keyword = re.sub(r'\W+', '', row_cells[col].value)
                illness_keywords[illness].append(keyword.lower())
                
    return illness_keywords